In [1]:
import os
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

import typing

In [2]:
def VGGNet(
    name: str,
    architecture: typing.List[ typing.Union[int, str] ],
    input_shape: typing.Tuple[int],
    classes: int = 1000
) -> tf.keras.Model:
    X_input = tf.keras.layers.Input(input_shape)

    X = make_conv_layer(X_input, architecture)

    X = tf.keras.layers.Flatten()(X)
    X = make_dense_layer(X, 4096)
    X = make_dense_layer(X, 4096)

    # classification layer
    X = tf.keras.layers.Dense(units = classes, activation = "softmax")(X)

    model = tf.keras.Model(inputs = X_input, outputs = X, name = name)
    return model

def make_conv_layer(
    X: tf.Tensor,
    architecture: typing.List[ typing.Union[int, str] ],
    activation: str = 'relu'
) -> tf.Tensor:

    for output in architecture:

        if type(output) == int:
            out_channels = output

            X = tf.keras.layers.Conv2D(
                filters = out_channels,
                kernel_size = (3, 3),
                strides = (1, 1),
                padding = "same"
            )(X)
            X = tf.keras.layers.BatchNormalization()(X)
            X = tf.keras.layers.Activation(activation)(X)
        else:
            X = tf.keras.layers.MaxPooling2D(
                pool_size = (2, 2),
                strides = (2, 2)
            )(X)

    return X

def make_dense_layer(X: tf.Tensor, output_units: int, dropout = 0.5, activation = 'relu') -> tf.Tensor:
    X = tf.keras.layers.Dense(units = output_units)(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Activation(activation)(X)
    X = tf.keras.layers.Dropout(dropout)(X)

    return X

In [3]:
VGG_types = {
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
}


vgg_model = VGGNet(name = "VGGNet16", architecture = VGG_types["VGG16"], input_shape=(224, 224, 3), classes = 8)
vgg_model.summary()

Model: "VGGNet16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 batch_normalization (BatchN  (None, 224, 224, 64)     256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 224, 224, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 batch_normalization_1 (Batc  (None, 224, 224, 64)     256       
 hNormalization)                                          

In [4]:
# Compile the model
vgg_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
batch_size = 32
epochs = 70

data_folder = 'data/Flowers'
flower_types = ["Babi", "Calimerio", "Chrysanthemum", "Hydrangeas", "Lisianthus", "Pingpong", "Rosy", "Tana"]

# Create empty lists to store image paths and corresponding labels
image_paths = []
labels = []

# Loop over the flower types and add image paths and labels to the lists
for i, flower_type in enumerate(flower_types):
    folder_path = os.path.join(data_folder, flower_type)
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.jpg'):
            image_path = os.path.join(folder_path, file_name)
            image_paths.append(image_path)
            labels.append(i)

# Shuffle the data
data = list(zip(image_paths, labels))
random.shuffle(data)
image_paths, labels = zip(*data)


# Split the data into training and validation sets
train_image_paths, val_image_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42)

# Define the data generators for the training and validation sets
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    data_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

validation_generator = val_datagen.flow_from_directory(
    data_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True,
    seed=42
)

Found 4621 images belonging to 8 classes.
Found 0 images belonging to 8 classes.


In [8]:
vgg_history = vgg_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/70
 8/72 [==>...........................] - ETA: 17:27 - loss: 3.9873 - accuracy: 0.1406

In [ ]:
plt.plot(vgg_history.history['accuracy'])
plt.plot(vgg_history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()